In [58]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R


def cam_calib(w, h, square_size, criteria, path,outpath):
    """
    说明：用来获得相机内参
    参数：
    w,h 棋盘格内角点个数
    square_size:棋盘格尺寸
    criteria：检测角点准则
    path：检测图像路径

    """
    list_path = os.listdir(path)

    # 准备棋盘格的世界坐标点
    objp = np.zeros((w * h, 3), np.float32)
    objp[:, :2] = np.mgrid[0:w, 0:h].T.reshape(-1, 2)
    objp *= square_size

    grays = []
    objpoints = []  # 世界坐标点
    imgpoints = []  # 图像像素点

    # 遍历图像并检测棋盘角点
    for file in list_path:
        file_name = os.path.join(path, file)
        img = cv2.imread(file_name)
        scale = 0.5  # 50%缩小
        img_resize = cv2.resize(
            img, (int(img.shape[1] * scale), int(img.shape[0] * scale)))

        gray = cv2.cvtColor(img_resize, cv2.COLOR_BGR2GRAY)
        # gray_resize = cv2.resize(gray, (int(gray.shape[1] * scale), int(gray.shape[0] * scale)))
        print('image pixel', gray.shape[::-1])
        grays.append(gray)

        ret, corners = cv2.findChessboardCorners(gray, (w, h), None)
        print(f"Chessboard found in {file}: {ret}")

        if ret:
            objpoints.append(objp)
            corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1),
                                        criteria)
            imgpoints.append(corners2)
            
            
            # 可视化检测到的角点（用 matplotlib 替代 OpenCV）
            img_with_corners = img_resize.copy()
            cv2.drawChessboardCorners(img_with_corners, (w, h), corners2, ret)
            cv2.imwrite(f'{outpath}\\{file}', img_with_corners)
            #使用 matplotlib 显示
            # plt.figure(figsize=(8, 6))
            # plt.imshow(cv2.cvtColor(img_with_corners, cv2.COLOR_BGR2RGB))  # 转换为 RGB 格式
            # plt.scatter(corners2[:, 0, 0], corners2[:, 0, 1], marker='o', c='red', s=10, label="Detected Corners")  # `s`控制点大小
            # plt.title(f"Detected Corners: {file}")
            # plt.rcParams['lines.linewidth'] = 2  # 设置线条宽度
            # plt.axis("off")  # 隐藏坐标轴
            # plt.show()


    # 相机标定
    print("\nStarting camera calibration...")
    # ret, camera_matrix, distortion_coeffs, rvecs, tvecs = cv2.calibrateCamera(
    #     objpoints, imgpoints, grays[0].shape[::-1], None, None
    # )
    ret, camera_matrix, distortion_coeffs, rvecs, tvecs = cv2.calibrateCamera(
        objpoints,
        imgpoints,
        grays[0].shape[::-1],
        None,
        None,
        flags=cv2.CALIB_RATIONAL_MODEL | cv2.CALIB_FIX_K3 | cv2.CALIB_FIX_K4
        | cv2.CALIB_FIX_K5)

    # 计算重投影误差
    mean_error = 0
    for i in range(len(objpoints)):
        imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i],
                                          camera_matrix, distortion_coeffs)
        error = cv2.norm(imgpoints[i].squeeze(), imgpoints2.squeeze(),
                         cv2.NORM_L2) / len(imgpoints2)
        mean_error += error

    print("\n总平均误差 (Mean Reprojection Error):", mean_error / len(objpoints))

    new_camera_matrix = camera_matrix.copy()
    new_camera_matrix[0, 0] /= scale  # fx
    new_camera_matrix[1, 1] /= scale  # fy
    new_camera_matrix[0, 2] /= scale  # cx
    new_camera_matrix[1, 2] /= scale  # cy
    # 打印结果
    print("相机矩阵 (Camera Matrix):\n", new_camera_matrix)
    print("\n畸变系数 (Distortion Coefficients):\n", distortion_coeffs)
    # 保存标定结果
    np.savez("calibration_result.npz",
             camera_matrix=new_camera_matrix,
             distortion_coeffs=distortion_coeffs,
             rvecs=rvecs,
             tvecs=tvecs)
    for file in list_path:
        file_name = os.path.join(path, file)
        img = cv2.imread(file_name)
        
    return new_camera_matrix, distortion_coeffs

In [80]:
import numpy as np
import cv2
import os

def corner_pixel_extract(file,file_name,output_path):
    """检测样品的四个角点，并返回角点的像素坐标"""
    points = []
    img = cv2.imread(file_name)  
    # 转为灰度图
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 应用高斯模糊（减少噪声）
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    # 二值化处理
    retval, threshold_img = cv2.threshold(blurred, 35, 255, cv2.THRESH_BINARY)
    # 形态学操作，去除小噪点
    kernel = np.ones((5, 5), np.uint8)
    thresh = cv2.morphologyEx(threshold_img, cv2.MORPH_CLOSE, kernel)
    # 轮廓检测
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        peri = cv2.arcLength(contour, True)  # 计算轮廓周长
        # print(peri)
        if peri> 100:
            approx = cv2.approxPolyDP(contour, 0.02 * peri, True)  # 轮廓逼近

            if len(approx) == 4:  # 只保留四边形
                print("检测到四边形，顶点坐标：")
                
                
                for point in approx:
                    x, y = point[0]
                    points.append([x, y])
                    print(f"({x}, {y})")
                    cv2.circle(img, (x, y), 5, (0, 0, 255), -1)  # 标记四边形顶点（红色）
                cv2.drawContours(img, [approx], -1, (0, 255, 0), 3)  # 绿色绘制四边形
            # 保存检测结果
    
                cv2.imwrite(f'{output_path}\\{file}', img)

                print(f"处理完成，结果已保存: {output_path}")
    return points

In [69]:
def object_points_calculate(T_ref, T_pose, point_board):
    """
    说明：计算某一位姿下棋盘格点在参考坐标系下的3d坐标
    T_ref:参考坐标系相对于base的变换矩阵
    T_pose该位姿下tcp相对于base的变换矩阵
    point_board：棋盘格点在tcp坐标系里的坐标
    """
    # 将3D点转换为齐次坐标
    origin_points_h = np.hstack(
        [point_board, np.ones((point_board.shape[0], 1))])
    # 使用矩阵乘法进行坐标变换
    #去掉齐次坐标中的最后一列，转换回普通的3D坐标
    object_points_base_inv = np.dot(T_pose, origin_points_h.T)
    object_points_o1 = np.dot(np.linalg.inv(T_ref), object_points_base_inv).T
    return object_points_o1[:, :3]


def kuka_to_transformation(pose):
    """将 KUKA 的点位信息转换为 4x4 变换矩阵"""
    # 平移向量
    t = np.array([pose[0], pose[1], pose[2]])

    # 计算旋转矩阵（ZYX顺序）
    rotation = R.from_euler('ZYX', [pose[3], pose[4], pose[5]],
                            degrees=True)  # KUKA的ABC通常以度为单位
    R_matrix = rotation.as_matrix()
    # print(R_matrix)

    # 构造4x4变换矩阵
    T = np.eye(4)
    T[:3, :3] = R_matrix
    T[:3, 3] = t
    # print('T', T)
    return T

def cam_solve( object_points, image_points, camera_matrix,
              distortion_coeffs):
    """调用pnp函数，返回rvec,tvec"""
    _, rvec, tvec, inliners = cv2.solvePnPRansac(object_points,
                                                 image_points.squeeze(),
                                                 camera_matrix,
                                                 distortion_coeffs,
                                                 reprojectionError=8.0,
                                                 confidence=0.99,
                                                 iterationsCount=150)
    if not _:
        print("PnP 求解失败")
    else:
        return rvec, tvec

In [61]:
class Points_pairs:
    """定义对象，储存2d-3d点对及其编号"""
    def __init__(self, obj_id, pixel_corners,world_coordinates=None):
        self.id = obj_id
        self.pixel_corners = pixel_corners
        self.world_coordinates = world_coordinates

    def __str__(self):
        return f"Object {self.id}: Pixel Corners={self.pixel_corners}, World Coordinates={self.world_coordinates}"

In [82]:
path = 'D:\\labAAA\\printingLight\\process\\camera_set\\test250313\\camsolveImages'
list_path = os.listdir(path)
output_path = 'D:\\labAAA\\printingLight\\process\\camera_set\\test250313\\pixels'
pixel_object_pair = []

for file in list_path:
    file_name = os.path.join(path, file)
    corners = corner_pixel_extract(file,file_name,output_path)
    if corners:
        pair_obj = Points_pairs(list_path.index(file),corners)
        pixel_object_pair.append(pair_obj)


检测到四边形，顶点坐标：
(3707, 1820)
(2652, 2063)
(3006, 3081)
(4045, 2884)
处理完成，结果已保存: D:\labAAA\printingLight\process\camera_set\test250313\pixels
检测到四边形，顶点坐标：
(3011, 1810)
(2014, 2067)
(2380, 3083)
(3384, 2887)
处理完成，结果已保存: D:\labAAA\printingLight\process\camera_set\test250313\pixels
检测到四边形，顶点坐标：
(3009, 1476)
(2013, 1749)
(2378, 2770)
(3382, 2560)
处理完成，结果已保存: D:\labAAA\printingLight\process\camera_set\test250313\pixels
检测到四边形，顶点坐标：
(3006, 1142)
(2011, 1428)
(2377, 2459)
(3383, 2242)
处理完成，结果已保存: D:\labAAA\printingLight\process\camera_set\test250313\pixels
检测到四边形，顶点坐标：
(3002, 808)
(2007, 1111)
(2373, 2148)
(3380, 1916)
处理完成，结果已保存: D:\labAAA\printingLight\process\camera_set\test250313\pixels
检测到四边形，顶点坐标：
(3002, 472)
(2007, 790)
(2372, 1835)
(3379, 1587)
处理完成，结果已保存: D:\labAAA\printingLight\process\camera_set\test250313\pixels
检测到四边形，顶点坐标：
(2666, 475)
(1690, 793)
(2060, 1838)
(3052, 1589)
处理完成，结果已保存: D:\labAAA\printingLight\process\camera_set\test250313\pixels
检测到四边形，顶点坐标：
(2670, 810)
(1692, 1111)
(

In [88]:
pose_list = [[-222.992, -1884.805, 1400, 32.616, 12.209, 70.802],
[-222.992, -1884.805, 1450, 32.616, 12.209, 70.802],
[-222.992, -1884.805, 1500, 32.616, 12.209, 70.802],
[-222.992, -1884.805, 1550, 32.616, 12.209, 70.802],
[-222.992, -1884.805, 1600, 32.616, 12.209, 70.802],
[-196.496, -1842.402, 1600, 32.616, 12.209, 70.802],
[-196.496, -1842.402, 1550, 32.616, 12.209, 70.802], 
[-196.496, -1842.402, 1500, 32.616, 12.209, 70.802], 
[-196.496, -1842.402, 1450, 32.616, 12.209, 70.802], 
[-196.496, -1842.402, 1400, 32.616, 12.209, 70.802], 
[-170, -1800, 1400, 32.616, 12.209, 70.802], 
[-170, -1800, 1450, 32.616, 12.209, 70.802], 
[-170, -1800, 1500, 32.616, 12.209, 70.802], 
[-170, -1800, 1550, 32.616, 12.209, 70.802], 
[-170, -1800, 1600, 32.616, 12.209, 70.802], 
[-143.504, -1757.598, 1600, 32.616, 12.209, 70.802], 
[-143.504, -1757.598, 1550, 32.616, 12.209, 70.802], 
[-143.504, -1757.598, 1500, 32.616, 12.209, 70.802], 
[-143.504, -1757.598, 1450, 32.616, 12.209, 70.802], 
[-143.504, -1757.598, 1400, 32.616, 12.209, 70.802], 
]




In [65]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100,
            0.0001)



camera_matrix, distortion_coeffs = cam_calib(
    w=9,
    h=11,
    square_size=45,
    criteria=criteria,
    path='D:\\labAAA\\printingLight\\process\\camera_set\\test250313\\chessboard4calib',
    outpath='D:\\labAAA\\printingLight\\process\\camera_set\\test250313\\chessboard_corner')

image pixel (2736, 1824)
Chessboard found in IMG_6575.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6576.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6577.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6578.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6579.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6580.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6581.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6582.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6583.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6584.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6585.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6586.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6587.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6588.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6589.JPG: True

Starting camera calibration...

总平均误差 (

In [90]:
translation_vec = np.array([22.054, -4.485, -48.792])
ref = [-170, -1800, 1500, 58, 0, 90]  #tcp在world里的位置
corner = np.array([[-90, 90, 0], [90, 90, 0], [90, -90, 0], [-90, -90, 0]],
                  dtype=np.float64) + translation_vec
rvec_list = []
tvec_list = []

T_ref = kuka_to_transformation(ref)
for i in range(len(pixel_object_pair)):
    id = pixel_object_pair[i].id
    pose = pose_list[id]
    T_pose = kuka_to_transformation(pose)
    world_coordinates = object_points_calculate(T_ref, T_pose, corner)
    pixel_object_pair[i].world_coordinates = world_coordinates
    # print(pixel_object_pair[i])

    rvec, tvec = cam_solve(
        object_points=np.array(world_coordinates,dtype=np.float64).reshape(-1, 1, 3),
        image_points=np.array(pixel_object_pair[i].pixel_corners,dtype=np.float64).reshape(-1, 2),
        camera_matrix=camera_matrix,
        distortion_coeffs=distortion_coeffs)
    # 计算旋转矩阵
    rotation_matrix, _ = cv2.Rodrigues(rvec)
    print("Rotation Matrix:\n", rotation_matrix)
    print("Translation Vector:\n", tvec)
    # rvec_list.append(rvec)
    # tvec_list.append(tvec)


# print(tvec_list)


Rotation Matrix:
 [[-0.9999224   0.00185221  0.01231901]
 [-0.00163057 -0.99983706  0.01797746]
 [ 0.0123503   0.01795598  0.9997625 ]]
Translation Vector:
 [[ 6.77065153e+01]
 [-1.39758726e+00]
 [ 1.61630617e+03]]
Rotation Matrix:
 [[-9.99664868e-01 -6.88463591e-04  2.58781205e-02]
 [ 2.04840610e-03 -9.98615555e-01  5.25621310e-02]
 [ 2.58061066e-02  5.25975246e-02  9.98282297e-01]]
Translation Vector:
 [[ -34.53118191]
 [  51.19231284]
 [1649.43707534]]
Rotation Matrix:
 [[-9.99714561e-01 -6.03301237e-05  2.38912666e-02]
 [ 1.41669834e-03 -9.98386864e-01  5.67597013e-02]
 [ 2.38493024e-02  5.67773466e-02  9.98101971e-01]]
Translation Vector:
 [[ -34.87146313]
 [  49.97867041]
 [1649.94222025]]
Rotation Matrix:
 [[-0.99936823  0.00347357  0.03537062]
 [-0.00153645 -0.99850456  0.05464698]
 [ 0.03550754  0.05455811  0.99787906]]
Translation Vector:
 [[ -35.06829525]
 [  48.34961019]
 [1641.49467631]]
Rotation Matrix:
 [[-0.99943565  0.00361756  0.0333959 ]
 [-0.00165529 -0.9982802   0.